# 1.0 Obtenção dos Dados

## 1.1 Conectando ao Database 

In [12]:
import psycopg2
import pandas as pd

pd.set_option('display.max_columns', 500)

# Defina os parâmetros de conexão
host = "34.55.27.249"  # ou o IP do seu servidor de banco de dados
dbname = "hackday-bookstore-cds"  # nome do seu banco de dados
user = "alunocds"  # seu nome de usuário
password = "hackday2025"  # sua senha


## 1.2 Leitura da tabela de Vendas [Sales]

In [13]:

try:
    # Estabelecendo a conexão
    conn = psycopg2.connect(
        host=host,
        dbname=dbname,
        user=user,
        password=password
    )

    # Criando um cursor para interagir com o banco
    cur = conn.cursor()

    # Definindo a consulta SQL completa (incluindo SET, CREATE, INSERT, e SELECT)
    sql_query = """
    SET datestyle = 'DMY';

    DROP TABLE IF EXISTS sales_corrected;

    CREATE TEMP TABLE sales_corrected (
        sale_date DATE,
        isbn TEXT,
        discount TEXT,
        item_id TEXT,
        order_id TEXT
    );

    INSERT INTO sales_corrected (sale_date, isbn, discount, item_id, order_id)
    SELECT 
        sale_date::DATE,
        isbn,
        discount,
        item_id,
        order_id
    FROM sales
    WHERE isbn IS NOT NULL AND discount IS NOT NULL;

    INSERT INTO sales_corrected (sale_date, isbn, discount, item_id, order_id)
    SELECT
        SPLIT_PART(sale_date, ',', 1)::DATE AS sale_date,
        SPLIT_PART(sale_date, ',', 2) AS isbn,
        SPLIT_PART(sale_date, ',', 3) AS discount,
        SPLIT_PART(sale_date, ',', 4) AS item_id,
        SPLIT_PART(sale_date, ',', 5) AS order_id
    FROM sales
    WHERE sale_date LIKE '%,%';

        select    sale_date,
            isbn, 
            COUNT(isbn) 
            from sales_corrected
             group by isbn,sale_date
    """
    # Executando a consulta SQL completa
    cur.execute(sql_query)

    # Obtendo os resultados da consulta
    data = cur.fetchall()

    # Obtendo o nome das colunas (cabeçalhos da tabela)
    columns = [desc[0] for desc in cur.description]

    # Transformando os dados em um DataFrame
    df_sales = pd.DataFrame(data, columns=columns)

    # Exibindo as primeiras linhas do DataFrame
    print(df_sales.head())

    # Fechando o cursor e a conexão
    cur.close()
    conn.close()

except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")

    sale_date               isbn  count
0  2193-07-19  989-28-229-1404-4      9
1  2193-08-22  989-28-79-16808-7      1
2  2193-07-19  989-28-229-0197-6      2
3  2193-08-03  989-28-79-51397-9      3
4  2193-08-14  989-28-79-45675-7      6


In [14]:
df_sales_group = df_sales.groupby('isbn', as_index=False)['count'].sum()


## 1.3 Leitura e join das demais tabela com seleção de Features 

In [15]:
try:
    # Estabelecendo a conexão
    conn = psycopg2.connect(
        host=host,
        dbname=dbname,
        user=user,
        password=password
    )

    # Criando um cursor para interagir com o banco
    cur = conn.cursor()

    # Definindo a consulta SQL completa (incluindo SET, CREATE, INSERT, e SELECT)
    sql_query = """
    SET datestyle = 'DMY';

    SELECT 
        ratings.book_id AS rating_book_id,
        ratings.rating,
        e.isbn,
        e.format_id,
        e.pub_id,
        e.publication_date,
        e.pages,
        e.print_run_size_k,
        e.price,
        p.name AS publisher_name,
        p.city AS publisher_city,
        p.state AS publisher_state,
        p.country AS publisher_country,
        p.year_established,
        p.marketing_spend,
        i.genre_id,
        i.series_id,
        i.volume_number,
        s.series_name,
        g.genre_desc,
        f.format_desc,
        b.title AS book_title,
        b.author_id,
        a."BookID" AS award_book_id,
        a."Title" AS award_title,
        a."Year Won",
        au.first_name AS author_first_name,
        au.last_name AS author_last_name,
        au.birthday AS author_birthday,
        au.country_residence AS author_country,
        au.hrs_writing_day AS author_hrs_writing_day
    FROM (  select book_id as book_id,
            avg(rating) as rating
            from ratings
            group by book_id ) as ratings
    LEFT JOIN edition e ON ratings.book_id = e.book_id
    LEFT JOIN publisher p ON p.book_id = ratings.book_id
    LEFT JOIN info i ON i.book_id = p.book_id
    LEFT JOIN series s ON s.series_id = i.series_id
    LEFT JOIN genders g ON g.genre_id = i.genre_id
    LEFT JOIN format f ON f.format_id = e.format_id
    LEFT JOIN book b ON i.book_id = b.book_id
    LEFT JOIN award a ON b.book_id = a."BookID"
    LEFT JOIN author au ON b.author_id = au.author_id;
    """

    # Executando a consulta SQL completa
    cur.execute(sql_query)

    # Obtendo os resultados da consulta
    data = cur.fetchall()

    # Obtendo o nome das colunas (cabeçalhos da tabela)
    columns = [desc[0] for desc in cur.description]

    # Transformando os dados em um DataFrame
    df = pd.DataFrame(data, columns=columns)

    # Fechando o cursor e a conexão
    cur.close()
    conn.close()

except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")


## 1.4 Tratamento de dados inicial


In [16]:
# Unifying author name into one feature

df['author_name'] = df['author_first_name'] + ' ' + df['author_last_name']
df.drop(columns=['author_first_name','author_last_name'], axis = 1, inplace = True)
df.drop_duplicates(inplace=True)

In [17]:
# Selecting the features according to the insights
df_filtered = df[['isbn','rating_book_id','book_title', 'rating','price','publisher_name',
                  'marketing_spend','genre_desc','series_name','format_desc','author_name','Year Won']]

In [18]:
# Ensure the 'rating' column is numeric and round to 2 decimal places
df_filtered = df_filtered.copy()
df_filtered['rating'] = pd.to_numeric(df_filtered['rating'], errors='coerce').round(2)

## 1.5 Unindo as tabelas 

In [19]:
# Merge df_grouped with df2 on 'isbn'
df_final = pd.merge(df_filtered, df_sales_group, on='isbn', how='left')

# Rename 'count' to 'sales'
df_final.rename(columns={'count': 'sales_count'}, inplace=True)
df_final.loc[:, 'sales_count'] = pd.to_numeric(df_final['sales_count'], errors='coerce')

# Add invoice feature based on sales and price
df_final['invoice'] = df_final['sales_count'] * df_final['price']

#Dropping NaN book_title rows 
df_final = df_final.dropna(subset=['book_title'])

df_final.head()

,isbn,rating_book_id,book_title,rating,price,publisher_name,marketing_spend,genre_desc,series_name,format_desc,author_name,Year Won,sales_count,invoice
0,989-28-654-4370-9,DE571,Don t Check your Ego,3.99,15.99,Palimpsest Printing,48000.0,Nonfiction,None,Trade paperback,R.M. Larner,NaN,45.0,719.55
1,989-28-654-9826-6,DE571,Don t Check your Ego,3.99,28.99,Palimpsest Printing,48000.0,Nonfiction,None,Hardcover,R.M. Larner,NaN,14.0,405.86
2,989-28-79-91028-0,TM925,The Mallemaroking,4.66,12.50,Etaoin Shrdlu Press,2320000.0,SciFi/Fantasy,The Mallemaroking Saga,Trade paperback,Bianca Thompson,2179.0,1368.0,17100.00
3,989-28-79-13801-1,TM925,The Mallemaroking,4.66,8.99,Etaoin Shrdlu Press,2320000.0,SciFi/Fantasy,The Mallemaroking Saga,Mass market paperback,Bianca Thompson,2179.0,4568.0,41066.32
4,989-28-79-71565-6,TM925,The Mallemaroking,4.66,21.50,Etaoin Shrdlu Press,2320000.0,SciFi/Fantasy,The Mallemaroking Saga,Hardcover,Bianca Thompson,2179.0,1279.0,27498.50


In [20]:
df = df_final.copy()

In [21]:
df.to_csv('data/full-data.csv', index = False)